In [17]:
import pandas as pd
pd.options.display.max_colwidth = 100


In [18]:
psdn_ocean_contract = '0x51Fa2efd62ee56a493f24AE963eAce7D0051929E'

# load data
psdn_ocean_data_file = '/Users/nvtrang/code/h2o/protocol-liquidity/data/export-token-0x51fa2efd62ee56a493f24ae963eace7d0051929e-psdnocean.csv'
psdn_ocean_transfers = pd.read_csv(psdn_ocean_data_file)

# types casting
psdn_ocean_transfers['DateTime'] = pd.to_datetime(psdn_ocean_transfers['DateTime'])
psdn_ocean_transfers["Quantity"] = [float(str(i).replace(",", "")) for i in psdn_ocean_transfers["Quantity"]]
psdn_ocean_transfers.head()

,Txhash,Blockno,UnixTimestamp,DateTime,From,To,Quantity,Method
0,0x6ced305c2f06ffbc267c3a83f7a3b735599741610ee83ea19c8757d4ba6fa399,15768074,1666012715,2022-10-17 13:18:35,0x0000000000000000000000000000000000000000,0x78e869870e425a4d78bc18c46be9a6b9b8417cb5,99.9,Deposit
1,0x542ff7199ac430b003fb65e8ca7224220517af1c4548c7025d36bd5377af73ba,15768378,1666016399,2022-10-17 14:19:59,0x78e869870e425a4d78bc18c46be9a6b9b8417cb5,0xba12222222228d8ba445958a75a0704d566bf2c8,0.8,Join Pool
2,0xf6967c4cf727ed79c616c286b31f7ad6338cf291de85059a7c59ee3989624c1f,15769054,1666024559,2022-10-17 16:35:59,0x0000000000000000000000000000000000000000,0x78e869870e425a4d78bc18c46be9a6b9b8417cb5,100.1,Deposit
3,0x854e1b6a368cffeba39dd57de6df31d5a351f0e0a86dcb417a21d27b0372c8d9,15769069,1666024739,2022-10-17 16:38:59,0x78e869870e425a4d78bc18c46be9a6b9b8417cb5,0xeb1cefa6f175b4c889582dfeea4ee155ccd5d2a5,100,Stake
4,0xa8b5006e723777c12dbbe086fd8e66d080c63825d6facb78b23ccd9f6cf56cb6,15769443,1666029239,2022-10-17 17:53:59,0x0000000000000000000000000000000000000000,0x1a7eedeadc8e38fadff8719f22d9ca54eda406a0,"24,975",Deposit


In [37]:
# user deposit Ocean and get psdnOcean
psdn_ocean_deposits = psdn_ocean_transfers.loc[(psdn_ocean_transfers["From"] == "0x0000000000000000000000000000000000000000") & (psdn_ocean_transfers["Method"] == "Deposit")]
psdn_ocean_deposits_per_day = psdn_ocean_deposits.groupby(pd.Grouper(key='DateTime', freq='D'))['Quantity'].sum().reset_index()

In [53]:
psdn_ocean_transfers.loc[ (psdn_ocean_transfers["Method"] == "Join Pool")].groupby(pd.Grouper(key='DateTime', freq='D'))['Quantity'].sum().reset_index()


,DateTime,Quantity
0,2022-10-17,0.800000
1,2022-10-18,275493.281000
2,2022-10-19,0.000000
3,2022-10-20,0.000000
4,2022-10-21,0.000000
...,...,...
153,2023-03-19,0.000000
154,2023-03-20,0.000000
155,2023-03-21,0.000000
156,2023-03-22,0.000000


In [82]:
import plotly.graph_objects as go

# Create traces
df = psdn_ocean_deposits_per_day
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['DateTime'], y=df['Quantity'],
                    mode='lines',
                    name='Ocean deposits'))

# df = psdn_ocean_transfers.loc[ (psdn_ocean_transfers["Method"] == "Join Pool")].groupby(pd.Grouper(key='DateTime', freq='D'))['Quantity'].sum().reset_index()
# fig.add_trace(go.Scatter(x=df['DateTime'], y=df['Quantity'],
#                     mode='lines',
#                     name='psdnOcean join pool'))

# df = psdn_ocean_transfers.loc[ (psdn_ocean_transfers["Method"] == "Stake")].groupby(pd.Grouper(key='DateTime', freq='D'))['Quantity'].sum().reset_index()
# fig.add_trace(go.Scatter(x=df['DateTime'], y=df['Quantity'],
#                     mode='lines',
#                     name='psdnOcean Stake'))

# Edit the layout
fig.update_layout(title='Ocean deposits per day',
                   xaxis_title='Day',
                   yaxis_title='Amount', 
                   autosize=True,
    )
fig.show()
fig.write_image("images/ocean_deposits_daily.png")
